In [1]:
import tracemalloc
import time
import scanpy as sc
import pandas as pd

import cupy as cp
import cupyx
import numpy as np
from tqdm import tqdm

from icecream import ic

from SEACells.core import SEACells


findfont: Font family ['Raleway'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Lato'] not found. Falling back to DejaVu Sans.


In [2]:
from scipy.sparse import csr_matrix

In [3]:
ad = sc.read("/home/aparna/DATA/aparnakumar/50000_cells/mouse_marioni_50k.h5ad") 
num_cells = 10000
ad = ad[:num_cells]  

In [4]:
## Core parameters 
# number of SEACells
n_SEACells = num_cells // 75
build_kernel_on = 'X_pca' # key in ad.obsm to use for computing metacells
                            # This would be replaced by 'X_svd' for ATAC data

## Additional parameters
n_waypoint_eigs = 10 # Number of eigenvalues to consider when initializing metacells

In [5]:
model2 = SEACells(ad, 
                 use_gpu=False, 
                 use_sparse=True, 
                 build_kernel_on=build_kernel_on, 
                 n_SEACells=n_SEACells, 
                 n_waypoint_eigs=n_waypoint_eigs,
                 convergence_epsilon = 1e-5)

model2.construct_kernel_matrix()
model2.initialize_archetypes()
model2.initialize()

SPARSE AND NOT GPU
TRYING SEACellsCPU
Welcome to SEACells!
Computing kNN graph using scanpy NN ...


In [ ]:
model4 = SEACells(ad, 
                 use_gpu=True, 
                 use_sparse=True, 
                 build_kernel_on=build_kernel_on, 
                 n_SEACells=n_SEACells, 
                 n_waypoint_eigs=n_waypoint_eigs,
                 convergence_epsilon = 1e-5)

model4.construct_kernel_matrix()
model4.initialize_archetypes()
model4.initialize()

SPARSE AND GPU
TRYING SEACellsGPU
Welcome to SEACells GPU!
build_graph.SEACellGraph completed
Computing kNN graph using scanpy NN ...
Computing radius for adaptive bandwidth kernel...



Making graph symmetric...
Parameter graph_construction = union being used to build KNN graph...
Computing RBF kernel...



Building similarity LIL matrix...



Constructing CSR matrix...
Building kernel on X_pca
Computing diffusion components from X_pca for waypoint initialization ... 
Determing nearest neighbor graph...


  0%|          | 0/25 [00:00<?, ?it/s]

Done.
Sampling waypoints ...
Done.
Selecting 118 cells from waypoint initialization.
Initializing residual matrix using greedy column selection
Initializing f and g...


/home/aparna/miniconda/envs/env_SEACells/lib/python3.8/site-packages/cupyx/scipy/sparse/_compressed.py:545: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive.
  warnings.warn('Changing the sparsity structure of a '
100%|██████████| 25/25 [00:01<00:00, 24.96it/s]


Selecting 15 cells from greedy initialization.
Randomly initialized A matrix.
Convergence threshold set to 0.0017789947495674043 based on epsilon = 1e-05


In [ ]:
model3 = SEACells(ad, 
                 use_gpu=True, 
                 use_sparse=False, 
                 build_kernel_on=build_kernel_on, 
                 n_SEACells=n_SEACells, 
                 n_waypoint_eigs=n_waypoint_eigs,
                 convergence_epsilon = 1e-5)

model3.construct_kernel_matrix()
model3.initialize_archetypes()
model3.initialize()

NOT SPARSE AND GPU
TRYING SEACellsGPUDense
Welcome to SEACells GPU!
Computing kNN graph using scanpy NN ...


Computing radius for adaptive bandwidth kernel...



Making graph symmetric...
Parameter graph_construction = union being used to build KNN graph...
Computing RBF kernel...



Building similarity LIL matrix...



Constructing CSR matrix...
Building kernel on X_pca
Computing diffusion components from X_pca for waypoint initialization ... 
Determing nearest neighbor graph...


  0%|          | 0/27 [00:00<?, ?it/s]

Done.
Sampling waypoints ...
Done.
Selecting 116 cells from waypoint initialization.
Initializing residual matrix using greedy column selection
Initializing f and g...


100%|██████████| 27/27 [00:00<00:00, 59.35it/s]


Selecting 17 cells from greedy initialization.
Randomly initialized A matrix.
Setting convergence threshold at 0.00177


In [ ]:
A0s = {'cpu': model2.A0, 'gpu_sparse': model4.A0, 'gpu_dense': model3.A0}
B0s = {'cpu': model2.B0, 'gpu_sparse': model4.B0, 'gpu_dense': model3.B0}

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns

def compute_sparsity(M):
    """
    Compute the proportion of non-zero elements in a matrix
    @param M: a dense numpy array
    """
    try: 
        sparsity = M.nnz / (M.shape[0] * M.shape[1])
    except Exception: 
        sparsity = np.count_nonzero(M) / M.size

    return sparsity

def check_results(d):
    """
    Check that all results in a dictionary are the same
    """
    
    v2 = d['cpu']
    print('Sparsity of {} is {}'.format('CPU', compute_sparsity(v2)))
    # Turn v2 into a numpy array
    v2 = v2.toarray()
    elt2 = np.where(v2.flatten() != 0)[0]

    for key in ['gpu_sparse', 'gpu_dense']:
        v1 = d[key]
        print('Sparsity of {} is {}'.format(key, compute_sparsity(v1)))
        # Convert to dense for comparison
        if key == 'gpu_sparse':
            v1 = cp.array(v1.todense())
        # Turn into a numpy array
        v1 = cp.asnumpy(v1)
        # Grab the nonzero elements of v1
        elt1 = np.where(v1.flatten() != 0)[0]
        # Check that the nonzero elements are the same
        try:
            assert set(list(elt1)) == set(list(elt2))
        except AssertionError:
            print('Nonzero elements of {} are different from nonzero elements of {}'.format('CPU', key))

In [ ]:
check_results(A0s)

Sparsity of CPU is 0.22032932330827068
Sparsity of gpu_sparse is 0.2205315789473684
Nonzero elements of CPU are different from nonzero elements of gpu_sparse
Sparsity of gpu_dense is 1.0
Nonzero elements of CPU are different from nonzero elements of gpu_dense


In [25]:
M = A0s['gpu_sparse']

print(type(M))
print(M.shape) 

matrix_np = M.get().toarray()

# Create axes labels 
cell_labels = ['Cell {}'.format(i) for i in range(M.shape[0])]
archetype_labels = ['Archetype {}'.format(i) for i in range(M.shape[1])]

# Create a dataframe
df = pd.DataFrame(matrix_np, index=cell_labels, columns=archetype_labels)

# Load the data into a dataframe
df = df.reset_index().melt(id_vars=['index'], value_vars=archetype_labels)
df.columns = ['Cell', 'Archetype', 'Value']

# Plot the dataframe
sns.set(style="whitegrid")
ax = sns.boxplot(x="Archetype", y="Value", data=df)


<class 'cupyx.scipy.sparse._csr.csr_matrix'>
(133, 10000)


In [ ]:
check_results(B0s)

Sparsity of CPU is 0.0001
Sparsity of gpu_sparse is 0.0001
Nonzero elements of CPU are different from nonzero elements of gpu_sparse
Sparsity of gpu_dense is 0.0001
Nonzero elements of CPU are different from nonzero elements of gpu_dense


In [ ]:
initial_As = {'cpu': model2.A_, 'gpu_sparse': model4.A_, 'gpu_dense': model3.A_}

In [ ]:
check_results(initial_As)

Sparsity of CPU is 0.29245413533834586
Sparsity of gpu_sparse is 0.4905052631578947
Nonzero elements of CPU are different from nonzero elements of gpu_sparse
Sparsity of gpu_dense is 0.2917609022556391
Nonzero elements of CPU are different from nonzero elements of gpu_dense


In [ ]:
def _updateA_cpu(self, B, A_prev):
    """Update step for assigment matrix A.

    Given archetype matrix B and using kernel matrix K, compute assignment matrix A using constrained gradient
    descent via Frank-Wolfe algorithm.

    :param B: (n x k csr_matrix) defining SEACells as weighted combinations of cells
    :param A_prev: (n x k csr_matrix) defining previous weights used for assigning cells to SEACells
    :return: (n x k csr_matrix) defining updated weights used for assigning cells to SEACells
    """
    n, k = B.shape
    A = A_prev

    t = 0  # current iteration (determine multiplicative update)

    # precompute some gradient terms
    t2 = (self.K @ B).T
    t1 = t2 @ B

    # update rows of A for given number of iterations
    while t < self.max_FW_iter:
        # compute gradient (must convert matrix to ndarray)
        G = 2.0 * np.array(t1 @ A - t2)

        # # get argmins - shape 1 x n
        amins = np.argmin(G, axis=0)
        amins = np.array(amins).reshape(-1)

        # # loop free implementation
        e = csr_matrix((np.ones(len(amins)), (amins, np.arange(n))), shape=A.shape)

        A += 2.0 / (t + 2.0) * (e - A)
        t += 1

    return A, t2, t1

In [ ]:
def _updateB_cpu(self, A, B_prev):
    """Update step for archetype matrix B.

    Given assignment matrix A and using kernel matrix K, compute archetype matrix B using constrained gradient
    descent via Frank-Wolfe algorithm.

    :param A: (n x k csr_matrix) defining weights used for assigning cells to SEACells
    :param B_prev: (n x k csr_matrix) defining previous SEACells as weighted combinations of cells
    :return: (n x k csr_matrix) defining updated SEACells as weighted combinations of cells
    """
    K = self.K
    k, n = A.shape

    B = B_prev

    # keep track of error
    t = 0

    # precompute some terms
    t1 = A @ A.T
    t2 = K @ A.T

    # update rows of B for a given number of iterations
    while t < self.max_FW_iter:
        # compute gradient (need to convert np.matrix to np.array)
        G = 2.0 * np.array(K @ B @ t1 - t2)

        # get all argmins
        amins = np.argmin(G, axis=0)
        amins = np.array(amins).reshape(-1)

        e = csr_matrix((np.ones(len(amins)), (amins, np.arange(k))), shape=B.shape)

        B += 2.0 / (t + 2.0) * (e - B)

        t += 1

    return B

In [ ]:
def _updateA_gpu(self, Bg, Ag):
    """Compute assignment matrix A using constrained gradient descent via Frank-Wolfe algorithm.

    Given archetype matrix B and using kernel matrix K, compute assignment matrix A using constrained gradient
    descent via Frank-Wolfe algorithm.

    :param B: (n x k csr_matrix) defining SEACells as weighted combinations of cells
    :param A_prev: (n x k csr_matrix) defining previous weights used for assigning cells to SEACells
    :return: (n x k csr_matrix) defining updated weights used for assigning cells to SEACells
    """
    n, k = Bg.shape
    t = 0  # current iteration (determine multiplicative update)
    Kg = self.K

    # precompute some gradient terms
    t2g = (Kg @ Bg).T
    t1g = t2g @ Bg

    # update rows of A for given number of iterations
    while t < self.max_FW_iter:
        Gg = 2.0*(t1g @ Ag - t2g)
        amins = Gg.argmin(axis = 0)
        
        eg = cp.zeros((k, n))
        eg[amins, cp.arange(n)] = 1.0
        eg = cupyx.scipy.sparse.csr_matrix(eg) 

        Ag += 2.0 / (t + 2.0) * (eg - Ag)
        t += 1
    
    return Ag, t2g, t1g

In [ ]:
def _updateB_gpu(self, Ag, Bg):
    """Compute archetype matrix B using constrained gradient descent via Frank-Wolfe algorithm.

    Given assignment matrix A and using kernel matrix K, compute archetype matrix B using constrained gradient
    descent via Frank-Wolfe algorithm.

    :param A: (n x k csr_matrix) defining weights used for assigning cells to SEACells
    :param B_prev: (n x k csr_matrix) defining previous SEACells as weighted combinations of cells
    :return: (n x k csr_matrix) defining updated SEACells as weighted combinations of cells
    """
    # ic("_updateB")
    k, n = Ag.shape

    # keep track of error
    t = 0

    Kg = self.K
    # precompute some terms
    t1g = Ag.dot(Ag.T)
    t2g = Kg.dot(Ag.T)

    # update rows of B for a given number of iterations
    while t < self.max_FW_iter:
        # compute gradient
        Gg = 2 * (Kg.dot(Bg).dot(t1g) -t2g)

        # get all argmins
        amins = Gg.argmin(axis=0)

        eg = cp.zeros((n, k))
        eg[amins, cp.arange(k)] = 1.0
        eg = cupyx.scipy.sparse.csr_matrix(eg)

        f = 2.0 / (t + 2.0)
        Bg = Bg + (f * (eg - Bg))

        t += 1


    del (
        t1g,
        t2g,
        Ag,
        Kg,
        Gg,
        eg,
        amins,
    )
    cp._default_memory_pool.free_all_blocks()

    return Bg

In [ ]:
A_cpu = model2.A_
B_cpu = model2.B_

In [ ]:
A_gpu = model4.A_
B_gpu = model4.B_

In [ ]:
K_gpu = model4.K 
K_cpu = model2.K

In [ ]:
ic(A_cpu.shape, B_cpu.shape, A_gpu.shape, B_gpu.shape, K_gpu.shape, K_cpu.shape)

ic| A_cpu.shape: (133, 10000)
    B_cpu.shape: (10000, 133)
    A_gpu.shape: (133, 10000)
    B_gpu.shape: (10000, 133)
    K_gpu.shape: (10000, 10000)
    K_cpu.shape: (10000, 10000)


((133, 10000),
 (10000, 133),
 (133, 10000),
 (10000, 133),
 (10000, 10000),
 (10000, 10000))

In [ ]:
# Print out the sparsity measures of the matrices 
ic(A_cpu.nnz / (A_cpu.shape[0] * A_cpu.shape[1]), 
   B_cpu.nnz / (B_cpu.shape[0] * B_cpu.shape[1]),
   A_gpu.nnz / (A_gpu.shape[0] * A_gpu.shape[1]), 
   B_gpu.nnz / (B_gpu.shape[0] * B_gpu.shape[1]), 
   K_cpu.nnz / (K_gpu.shape[0] * K_gpu.shape[1]),
   K_gpu.nnz / (K_cpu.shape[0] * K_cpu.shape[1]))

ic| A_cpu.nnz / (A_cpu.shape[0] * A_cpu.shape[1]): 0.29245413533834586
    B_cpu.nnz / (B_cpu.shape[0] * B_cpu.shape[1]): 0.0001
    A_gpu.nnz / (A_gpu.shape[0] * A_gpu.shape[1]): 0.4905052631578947
    B_gpu.nnz / (B_gpu.shape[0] * B_gpu.shape[1]): 0.0001
    K_cpu.nnz / (K_gpu.shape[0] * K_gpu.shape[1]): 0.01970384
    K_gpu.nnz / (K_cpu.shape[0] * K_cpu.shape[1]): 0.01970384


(0.29245413533834586,
 0.0001,
 0.4905052631578947,
 0.0001,
 0.01970384,
 0.01970384)

In [ ]:
# Check the types of the matrices 
ic(type(A_cpu), type(B_cpu), type(A_gpu), type(B_gpu), type(K_cpu), type(K_gpu))

ic| type(A_cpu): <class 'scipy.sparse.csc.csc_matrix'>
    type(B_cpu): <class 'scipy.sparse.csr.csr_matrix'>
    type(A_gpu): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
    type(B_gpu): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
    type(K_cpu): <class 'scipy.sparse.csr.csr_matrix'>
    type(K_gpu): <class 'cupyx.scipy.sparse._csr.csr_matrix'>


(scipy.sparse.csc.csc_matrix,
 scipy.sparse.csr.csr_matrix,
 cupyx.scipy.sparse._csr.csr_matrix,
 cupyx.scipy.sparse._csr.csr_matrix,
 scipy.sparse.csr.csr_matrix,
 cupyx.scipy.sparse._csr.csr_matrix)

In [ ]:
# Check the data types of the matrices 
ic(A_cpu.dtype, B_cpu.dtype, A_gpu.dtype, B_gpu.dtype, K_cpu.dtype, K_gpu.dtype)

ic| A_cpu.dtype: dtype('float64')
    B_cpu.dtype: dtype('float64')
    A_gpu.dtype: dtype('float64')


    B_gpu.dtype: dtype('float64')
    K_cpu.dtype: dtype('float64')
    K_gpu.dtype: dtype('float64')


(dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'))

In [ ]:
# Compare K_cpu and K_gpu

diff = K_cpu - K_gpu.get()

# Create a frequency table of the differences 
ic(pd.value_counts(diff.toarray().flatten()))

ic| pd.value_counts(diff.toarray().flatten()): 0.0    100000000
                                               dtype: int64


0.0    100000000
dtype: int64

In [ ]:
# Set A_gpu and B_gpu to be the same as A_cpu and B_cpu, but with the same type as A_gpu and B_gpu 
A_gpu = cupyx.scipy.sparse.csr_matrix(A_cpu)
B_gpu = cupyx.scipy.sparse.csr_matrix(B_cpu)

# Set model4.K to be the same as model2.K, but with the same type as model4.K
model4.K = cupyx.scipy.sparse.csr_matrix(model2.K)

In [ ]:
# check if the K matrices are the same 

diff = model2.K - model4.K.get()

# Create a frequency table of the differences
ic(pd.value_counts(diff.toarray().flatten()))

ic| pd.value_counts(diff.toarray().flatten()): 0.0    100000000
                                               dtype: int64


0.0    100000000
dtype: int64

In [ ]:
# Check if the A matrices are the same 
diff = A_cpu - A_gpu.get()

# Create a frequency table of the differences
ic(pd.value_counts(diff.toarray().flatten()))


ic| pd.value_counts

(diff.toarray().flatten()): 0.0    1330000
                                               dtype: int64


0.0    1330000
dtype: int64

In [ ]:
# Check if the B matrices are the same 
diff = B_cpu - B_gpu.get()

# Create a frequency table of the differences
ic(pd.value_counts(diff.toarray().flatten()))

ic| pd.value_counts(diff.toarray(

).flatten()): 0.0    1330000
                                               dtype: int64


0.0    1330000
dtype: int64

In [ ]:
t2 = (model2.K @ B_cpu).T
t1 = t2 @ B_cpu 

t2g = (model4.K @ B_gpu).T 
t1g = t2g @ B_gpu

# Check if the t2 matrices are the same
diff = t2 - t2g.get()

# Create a frequency table of the differences
ic(pd.value_counts(diff.toarray().flatten()))

# Check if the t1 matrices are the same
diff = t1 - t1g.get()

# Create a frequency table of the differences
ic(pd.value_counts(diff.toarray().flatten()))


ic| pd.value_counts(diff.toarray().flatten()): 0.0    1330000
                                               dtype: int64
ic| pd.value_counts(diff.toarray().flatten()): 0.0    17689
                                               dtype: int64


0.0    17689
dtype: int64

In [ ]:
# Check the data types of the t2, t2g, t1, t1g matrices

ic(t2.dtype, t2g.dtype, t1.dtype, t1g.dtype)

ic| t2.dtype: dtype('float64')
    t2g.dtype: dtype('float64')
    t1.dtype: dtype('float64')
    t1g.dtype: dtype('float64')


(dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'))

In [ ]:
G = t1 @ A_cpu 
Gg = cupyx.scipy.sparse.csr_matrix(t1) @ cupyx.scipy.sparse.csr_matrix(A_cpu)

ic(type(G), type(Gg))

ic(G)
ic(Gg)

# Check if the G matrices are the same
diff = G - Gg.get()

ic(type(diff))
ic(Gg.data.dtype)


# Create a frequency table of the differences
ic(pd.value_counts(diff.toarray().flatten()))

ic| type(G): <class 'scipy.sparse.csc.csc_matrix'>


    type(Gg): <class 'cupyx.scipy.sparse._csr.csr_matrix'>
ic| G: <133x10000 sparse matrix of type '<class 'numpy.float64'>'
       	with 1052040 stored elements in Compressed Sparse Column format>
ic| Gg: <cupyx.scipy.sparse._csr.csr_matrix object at 0x7fc6989504f0>
ic| type(diff): <class 'scipy.sparse.csc.csc_matrix'>
ic| Gg.data.dtype: dtype('float64')
ic| pd.value_counts(diff.toarray().flatten()):  0.000000e+00    1314013
                                               -3.469447e-18       3240
                                                3.469447e-18       2893
                                                1.734723e-18       2583
                                               -1.734723e-18       2250
                                               -8.673617e-19       1780
                                                6.938894e-18       1053
                                               -6.938894e-18        415
                                                5.551115e-17     

 0.000000e+00    1314013
-3.469447e-18       3240
 3.469447e-18       2893
 1.734723e-18       2583
-1.734723e-18       2250
-8.673617e-19       1780
 6.938894e-18       1053
-6.938894e-18        415
 5.551115e-17        230
-5.551115e-17        227
-2.775558e-17        211
 2.775558e-17        191
 1.110223e-16        150
-1.110223e-16        148
-1.387779e-17        146
 1.387779e-17        145
 2.220446e-16        143
-2.220446e-16        116
 4.440892e-16         32
-4.440892e-16         30
 8.673617e-19          2
-8.881784e-16          2
dtype: int64

In [ ]:
# Run updateA on CPU and GPU and compare the results 

# CPU
A_cpu_2, t2, t1 = _updateA_cpu(model2, B_cpu, A_cpu)

# GPU
A_gpu_2, t2g, t1g = _updateA_gpu(model4, B_gpu, A_gpu)

In [ ]:
# Compare the sparsity of A_cpu_2 and A_gpu_2 
ic(A_cpu_2.nnz / (A_cpu_2.shape[0] * A_cpu_2.shape[1]), 
   A_gpu_2.nnz / (A_gpu_2.shape[0] * A_gpu_2.shape[1]))

ic| A_cpu_2.nnz / (A_cpu_2.shape[0] * A_cpu_2.shape[1]): 0.2944
    A_gpu_2.nnz / (A_gpu_2.shape[0] * A_gpu_2.shape[1]

): 0.40322706766917293


(0.2944, 0.40322706766917293)

In [ ]:
# Compare the values of A_cpu_2 and A_gpu_2 
diff = A_cpu_2 - A_gpu_2.get() 

ic(type(diff))

# Create a frequency table of the differences 
ic(pd.value_counts(diff.toarray().flatten()))

ic| 

type(diff): <class 'scipy.sparse.csc.csc_matrix'>
ic| pd.value_counts(diff.toarray().flatten()):  0.000000    825568
                                               -0.005490     12525
                                               -0.005490     12492
                                               -0.005490     11391
                                               -0.007843     11141
                                                             ...  
                                               -0.000784         1
                                               -0.007059         1
                                                0.025098         1
                                                0.000784         1
                                               -0.002353         1
                                               Length: 481, dtype: int64


 0.000000    825568
-0.005490     12525
-0.005490     12492
-0.005490     11391
-0.007843     11141
              ...  
-0.000784         1
-0.007059         1
 0.025098         1
 0.000784         1
-0.002353         1
Length: 481, dtype: int64

In [ ]:
# Compare t1 and t1g 
diff = t1 - t1g.get()

# Create a frequency table of the differences
ic(pd.value_counts(diff.toarray().flatten()))

ic| pd.value_counts(diff.toarray().flatten()): 0.0    17689
                                               dtype: int64


0.0    17689
dtype: int64

In [ ]:
# Compare t2 and t2g 

diff = t2 - t2g.get()

# Create a frequency table of the differences
ic(pd.value_counts(diff.toarray().flatten()))

ic| pd.value_counts(diff.

toarray().flatten()): 0.0    1330000
                                               dtype: int64


0.0    1330000
dtype: int64

In [ ]:
# Run updateB on CPU and GPU and compare the results

# CPU
B_cpu_2 = _updateB_cpu(model2, A_cpu, B_cpu)

# GPU
B_gpu_2 = _updateB_gpu(model4, A_gpu, B_gpu)

# Compare the sparsity of B_cpu_2 and B_gpu_2
ic(B_cpu_2.nnz / (B_cpu_2.shape[0] * B_cpu_2.shape[1]), 
   B_gpu_2.nnz / (B_gpu_2.shape[0] * B_gpu_2.shape[1]))

ic| B_cpu_2.nnz / (B_cpu_2.shape[0] * B_cpu_2.shape[1]): 0.0013045112781954887
    B_gpu_2.nnz / (B_gpu_2.shape[0] * B_gpu_2.shape[1]): 0.0013097744360902257


(0.0013045112781954887, 0.0013097744360902257)

In [ ]:
# Compare the values of B_cpu_2 and B_gpu_2
diff = B_cpu_2 - B_gpu_2.get()

ic(type(diff))

ic| type(diff): 

<class 'scipy.sparse.csr.csr_matrix'>


scipy.sparse.csr.csr_matrix

In [ ]:
# Create a frequency table of the differences 
ic(pd.value_counts(diff.toarray().flatten()))

ic| pd

.value_counts(diff.toarray().flatten()): 0.0    1330000
                                               dtype: int64


0.0    1330000
dtype: int64